In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps

from scipy.sparse import *

In [2]:
import time

In [101]:
from sklearn.metrics import roc_auc_score

# BPR

In [3]:
urm_path = '../../content/data_train.csv'

urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]

In [4]:
urm_all_df.head(10)

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
5,1,187,1.0
6,1,205,1.0
7,1,222,1.0
8,1,237,1.0
9,1,354,1.0


In [5]:
print ("The number of interactions is {}".format(len(urm_all_df)))

The number of interactions is 478730


In [6]:
userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024

Average interactions per user 37.88
Average interactions per item 21.54

Sparsity 99.83 %


Not all users are in the urm:

In [7]:
mapped_id, original_id = pd.factorize(urm_all_df["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(urm_all_df["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

We now replace the IDs in the dataframe and we are ready to use the data

In [8]:
urm_all_df["UserID"] = urm_all_df["UserID"].map(user_original_ID_to_index)
urm_all_df["ItemID"] = urm_all_df["ItemID"].map(item_original_ID_to_index)

In [9]:
urm_all_df.head(n=10)

,UserID,ItemID,Interaction
0,0,0,1.0
1,0,1,1.0
2,0,2,1.0
3,0,3,1.0
4,0,4,1.0
5,0,5,1.0
6,0,6,1.0
7,0,7,1.0
8,0,8,1.0
9,0,9,1.0


In [10]:
urm_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

urm_all

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [11]:
urm_all.tocsr()

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
def precision(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [13]:
def evaluate_algorithm(URM_test, recommender_object, at=5):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

            recommended_items = recommender_object.recommend(user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP))

---

# Splitting Train and Validation set

In [14]:
def train_test_split_urm(urm, train_percentage=0.8, seed=None):
    """
    Splits the User-Item Interaction Matrix (URM) into training and testing sets.

    Parameters:
    - urm (csr_matrix): The User-Item Interaction Matrix.
    - train_percentage (float): The percentage of interactions to include in the training set.
    - seed (int): Seed for reproducibility.

    Returns:
    - train_urm (csr_matrix): Training User-Item Interaction Matrix.
    - test_urm (csr_matrix): Testing User-Item Interaction Matrix.
    """

    if not (0 < train_percentage < 1):
        raise ValueError("train_percentage must be between 0 and 1 exclusive")

    # Set seed for reproducibility
    if seed is not None:
        np.random.seed(seed)

    # Get the non-zero indices (user and item indices)
    non_zero_indices = urm.nonzero()

    # Get the number of non-zero interactions
    num_interactions = len(non_zero_indices[0])

    # Randomly shuffle the indices
    shuffled_indices = np.arange(num_interactions)
    np.random.shuffle(shuffled_indices)

    # Calculate the number of interactions for the training set
    num_train_interactions = int(train_percentage * num_interactions)

    # Split the indices into training and testing sets
    train_indices = shuffled_indices[:num_train_interactions]
    test_indices = shuffled_indices[num_train_interactions:]

    # Create masks for indexing
    train_mask = np.zeros(num_interactions, dtype=bool)
    test_mask = np.zeros(num_interactions, dtype=bool)

    train_mask[train_indices] = True
    test_mask[test_indices] = True

    # Extract values and create new matrices
    train_data = urm.data[train_mask]
    train_rows = non_zero_indices[0][train_mask]
    train_cols = non_zero_indices[1][train_mask]

    test_data = urm.data[test_mask]
    test_rows = non_zero_indices[0][test_mask]
    test_cols = non_zero_indices[1][test_mask]

    # Create new matrices using the extracted values
    train_urm = coo_matrix((train_data, (train_rows, train_cols)), shape=urm.shape).tocsr()
    test_urm = coo_matrix((test_data, (test_rows, test_cols)), shape=urm.shape).tocsr()

    return train_urm, test_urm


Split the dataset into train and validation. We set the split to 0.8-0.2 as a standard. It's an hyperparameter: we should fine tune it.

In [16]:
# Assuming urm is your User-Item Interaction Matrix in csr_matrix format
train_urm, test_urm = train_test_split_urm(urm_all, train_percentage=0.8, seed=42)

-------------------------------------------------------------------------------

# BRP implementation #1

## Step 1: We create a dense similarity matrix, initialized as zero

In [17]:
n_users, n_items = train_urm.shape

In [ ]:
item_item_S = np.zeros((n_items, n_items), dtype = np.float32)
item_item_S

## Step 2: We sample a triplet

#### Create a mask of positive interactions. How to build it depends on the data

In [23]:
URM_mask = train_urm.copy()
URM_mask.data[URM_mask.data < 1.0] = 0

URM_mask.eliminate_zeros()
URM_mask

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>

In [24]:
user_id = np.random.choice(n_users)
user_id

7001

### Get user seen items and choose one

In [29]:
user_seen_items = URM_mask.indices[URM_mask.indptr[user_id]:URM_mask.indptr[user_id+1]]
user_seen_items

array([   87,  1946,  2433,  3330,  5086,  6903, 11507, 15193, 15617,
       16759, 20931])

In [30]:
pos_item_id = np.random.choice(user_seen_items)
pos_item_id

15193

### To select a negative item it's faster to just try again then to build a mapping of the non-seen items

In [31]:
neg_item_selected = False

# It's faster to just try again then to build a mapping of the non-seen items
while (not neg_item_selected):
    neg_item_id = np.random.randint(0, n_items)

    if (neg_item_id not in user_seen_items):
        neg_item_selected = True

neg_item_id

4972

## Step 2 - Computing prediction

#### The prediction depends on the model: SLIM, Matrix Factorization... 
#### Note that here the data is implicit so we do not multiply for the user rating, because it is always 1, we just sum the similarities of the seen items.

In [ ]:
x_ui = item_item_S[pos_item_id, user_seen_items].sum()
x_uj = item_item_S[neg_item_id, user_seen_items].sum()

print("x_ui is {:.4f}, x_uj is {:.4f}".format(x_ui, x_uj))

## Step 3 - Computing gradient

#### The gradient depends on the objective function: RMSE, BPR... 

In [33]:
x_uij = x_ui - x_uj
x_uij

0.0

#### The original BPR paper uses the logarithm of the sigmoid of x_ij, whose derivative is the following

In [34]:
sigmoid_item = 1 / (1 + np.exp(x_uij))
sigmoid_item

0.5

## Step 4 - Update model

#### How to update depends on the model itself, here we have just one paramether, the similarity matrix, so we perform just one update. In matrix factorization we have two.

#### We need a learning rate, which influences how fast the model will change. Small ones lead to slower convergence but often higher results

In [35]:
learning_rate = 1e-3

item_item_S[pos_item_id, user_seen_items] += learning_rate * sigmoid_item
item_item_S[pos_item_id, pos_item_id] = 0

item_item_S[neg_item_id, user_seen_items] -= learning_rate * sigmoid_item
item_item_S[neg_item_id, neg_item_id] = 0

#### Usually there is no relevant change in the scores over a single iteration

In [ ]:
x_i = item_item_S[pos_item_id, user_seen_items].sum()
x_j = item_item_S[neg_item_id, user_seen_items].sum()

print("x_i is {:.4f}, x_j is {:.4f}".format(x_i, x_j))

## Now we put everything in a training loop

In [18]:
def sample_triplet():
    
    non_empty_user = False
    
    while not non_empty_user:
        user_id = np.random.choice(n_users)
        user_seen_items = URM_mask.indices[URM_mask.indptr[user_id]:URM_mask.indptr[user_id+1]]
        
        if len(user_seen_items)>0:
            non_empty_user = True

    pos_item_id = np.random.choice(user_seen_items)

    neg_item_selected = False

    # It's faster to just try again then to build a mapping of the non-seen items
    while (not neg_item_selected):
        neg_item_id = np.random.randint(0, n_items)

        if (neg_item_id not in user_seen_items):
            neg_item_selected = True

    return user_id, pos_item_id, neg_item_id    

In [39]:
def train_one_epoch(item_item_S, learning_rate):

    start_time = time.time()
    for sample_num in range(n_users):

        # Sample triplet
        user_id, pos_item_id, neg_item_id = sample_triplet()
        
        user_seen_items = URM_mask.indices[URM_mask.indptr[user_id]:URM_mask.indptr[user_id+1]]

        # Prediction
        x_ui = item_item_S[pos_item_id, user_seen_items].sum()
        x_uj = item_item_S[neg_item_id, user_seen_items].sum()
        
        # Gradient
        x_uij = x_ui - x_uj

        sigmoid_item = 1 / (1 + np.exp(x_uij))
        
        # Update
        item_item_S[pos_item_id, user_seen_items] += learning_rate * sigmoid_item
        item_item_S[pos_item_id, pos_item_id] = 0

        item_item_S[neg_item_id, user_seen_items] -= learning_rate * sigmoid_item
        item_item_S[neg_item_id, neg_item_id] = 0

        # Print some stats
        if (sample_num +1)% 50000 == 0 or (sample_num +1) == n_users:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds. Samples per second {:.2f}".format(sample_num+1, elapsed_time, samples_per_second))

            
    return item_item_S, samples_per_second

In [43]:
learning_rate = 1e-6
    
item_item_S = np.zeros((n_items, n_items), dtype = np.float32)

for n_epoch in range(5):
    item_item_S, samples_per_second = train_one_epoch(item_item_S, learning_rate)

Iteration 12638 in 2.81 seconds. Samples per second 4505.19
Iteration 12638 in 2.99 seconds. Samples per second 4226.43
Iteration 12638 in 2.86 seconds. Samples per second 4415.45
Iteration 12638 in 2.81 seconds. Samples per second 4493.56
Iteration 12638 in 2.35 seconds. Samples per second 5372.89


In [44]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 14889.55 seconds, or 248.16 minutes


---

# BPR for MF

### What do we need for BPRMF?

* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience

## Step 1: We create the dense latent factor matrices

In [19]:
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [63]:
user_factors

array([[0.66603822, 0.34234408, 0.26885119, ..., 0.68222426, 0.03069365,
        0.70669693],
       [0.39181486, 0.02270246, 0.33772317, ..., 0.13306345, 0.84809781,
        0.18483642],
       [0.89932515, 0.38923883, 0.80725821, ..., 0.65292899, 0.40978492,
        0.86887452],
       ...,
       [0.83956817, 0.01545866, 0.9405424 , ..., 0.1580337 , 0.01185825,
        0.64508283],
       [0.47233957, 0.08436038, 0.00642065, ..., 0.42011993, 0.87053603,
        0.87153998],
       [0.68686646, 0.39929753, 0.07293899, ..., 0.94814257, 0.76425189,
        0.2678286 ]])

In [62]:
item_factors

array([[0.69877441, 0.55061747, 0.89386392, ..., 0.71118194, 0.28472001,
        0.00548145],
       [0.74841939, 0.54553185, 0.77199201, ..., 0.85406021, 0.04182174,
        0.66459814],
       [0.06142269, 0.69394203, 0.03509446, ..., 0.26863228, 0.98594394,
        0.76985901],
       ...,
       [0.08186662, 0.12896667, 0.22677384, ..., 0.25242397, 0.34674745,
        0.13218046],
       [0.04376521, 0.94744183, 0.47014952, ..., 0.71712699, 0.81341576,
        0.05479204],
       [0.1569719 , 0.05234645, 0.67173824, ..., 0.87863044, 0.26303987,
        0.45036299]])

## Step 2 - Computing prediction

In [25]:
user_id, pos_item_id, neg_item_id = sample_triplet()
(user_id, pos_item_id, neg_item_id)

(11038, 323, 2416)

In [26]:
x_ui = np.dot(user_factors[user_id,:], item_factors[pos_item_id,:])
x_uj = np.dot(user_factors[user_id,:], item_factors[neg_item_id,:])

print("x_ui is {:.4f}, x_uj is {:.4f}".format(x_ui, x_uj))

x_ui is 3.1880, x_uj is 3.0705


## Step 3 - Computing gradient

In [27]:
x_uij = x_ui - x_uj
x_uij

0.11752732716724257

In [28]:
sigmoid_item = 1 / (1 + np.exp(x_uij))
sigmoid_item

0.4706519416940661

## Step 4 - Update model

In [40]:
regularization = 1e-4
learning_rate = 1e-2

H_i = item_factors[pos_item_id,:]
H_j = item_factors[neg_item_id,:]
W_u = user_factors[user_id,:]


user_factors[user_id,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
item_factors[pos_item_id,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
item_factors[neg_item_id,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)

In [41]:
x_ui = np.dot(user_factors[user_id,:], item_factors[pos_item_id,:])
x_uj = np.dot(user_factors[user_id,:], item_factors[neg_item_id,:])

print("x_i is {:.4f}, x_j is {:.4f}".format(x_ui, x_uj))

x_i is 2.9261, x_j is 1.6352


In [42]:
x_uij = x_ui - x_uj
x_uij

1.2908183186357953

In [43]:
def train_one_epoch(user_factors, item_factors, learning_rate):

    start_time = time.time()
    for sample_num in range(n_users):

        # Sample triplet
        user_id, pos_item_id, neg_item_id = sample_triplet()
        
        # Prediction
        x_ui = np.dot(user_factors[user_id,:], item_factors[pos_item_id,:])
        x_uj = np.dot(user_factors[user_id,:], item_factors[neg_item_id,:])
        
        # Gradient
        x_uij = x_ui - x_uj

        sigmoid_item = 1 / (1 + np.exp(x_uij))
                
        H_i = item_factors[pos_item_id,:]
        H_j = item_factors[neg_item_id,:]
        W_u = user_factors[user_id,:]


        user_factors[user_id,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
        item_factors[pos_item_id,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
        item_factors[neg_item_id,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)

        # Print some stats
        if (sample_num +1)% 50000 == 0 or (sample_num +1) == n_users:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds. Samples per second {:.2f}".format(sample_num+1, elapsed_time, samples_per_second))
        
    return user_factors, item_factors, samples_per_second

In [95]:
learning_rate = 1e-4
num_factors = 10

#user_factors = np.random.random((n_users, num_factors))
#item_factors = np.random.random((n_items, num_factors))

for n_epoch in range(5):
    user_factors, item_factors, samples_per_second = train_one_epoch(user_factors, item_factors, learning_rate)

Iteration 12638 in 1.41 seconds. Samples per second 8987.93
Iteration 12638 in 1.37 seconds. Samples per second 9204.06
Iteration 12638 in 1.33 seconds. Samples per second 9515.75
Iteration 12638 in 1.33 seconds. Samples per second 9537.44
Iteration 12638 in 1.35 seconds. Samples per second 9340.02


---

# AUC Score
We use the AUC score to know how we're doing

In [85]:
# List of tuples (user_id, positive_movie) and (user_id, negative_movie)
num_samples = 1000  # Adjust the number of samples based on your needs

positive_samples = []
negative_samples = []

for _ in range(num_samples):
    user_id, pos_item_id, neg_item_id = sample_triplet()
    
    positive_samples.append((user_id, pos_item_id))  # Label 1 for positive samples
    negative_samples.append((user_id, neg_item_id))  # Label 0 for negative samples



In [46]:
positive_samples

[(6870, 1187),
 (9652, 1950),
 (1958, 10825),
 (4659, 1951),
 (7404, 2316),
 (8466, 52),
 (81, 128),
 (11153, 5326),
 (5767, 206),
 (5182, 808),
 (10616, 16317),
 (4588, 13454),
 (4536, 882),
 (12319, 952),
 (10470, 4504),
 (7496, 852),
 (12592, 4356),
 (1644, 50),
 (10833, 2981),
 (4254, 319),
 (2960, 6448),
 (9560, 5080),
 (10451, 5591),
 (6959, 558),
 (10996, 2444),
 (10930, 952),
 (6582, 7880),
 (10687, 4),
 (9826, 2486),
 (12189, 13380),
 (12037, 1293),
 (1200, 600),
 (11395, 6400),
 (783, 814),
 (4128, 100),
 (1289, 2463),
 (6648, 227),
 (1588, 3730),
 (514, 4926),
 (2618, 628),
 (4069, 118),
 (10672, 14272),
 (119, 3114),
 (6676, 15704),
 (10009, 88),
 (6793, 813),
 (8635, 15478),
 (8272, 117),
 (2914, 86),
 (2564, 2590),
 (5351, 4748),
 (1131, 3707),
 (2269, 55),
 (12386, 83),
 (1131, 2079),
 (11444, 21872),
 (10291, 1264),
 (10450, 8600),
 (177, 57),
 (7991, 3250),
 (11333, 1100),
 (4267, 4500),
 (702, 9195),
 (8404, 3705),
 (1473, 859),
 (6096, 14550),
 (6356, 3848),
 (11158,

In [36]:
negative_samples

[(1968, 6507),
 (10564, 9436),
 (10601, 18552),
 (3536, 5216),
 (3701, 13494),
 (1397, 9362),
 (10874, 20361),
 (10943, 1110),
 (2837, 14684),
 (9817, 2465),
 (5414, 3485),
 (5934, 21001),
 (936, 8336),
 (9880, 14764),
 (6752, 17581),
 (9112, 17665),
 (10470, 7835),
 (4795, 13168),
 (6870, 9406),
 (2489, 9272),
 (11775, 2853),
 (9475, 7304),
 (5128, 18932),
 (1085, 18373),
 (617, 1917),
 (8757, 2127),
 (349, 15440),
 (2233, 3068),
 (2357, 18794),
 (11443, 9576),
 (702, 7853),
 (861, 17360),
 (6389, 3090),
 (4789, 8330),
 (3114, 12598),
 (5296, 12579),
 (5054, 1278),
 (549, 9435),
 (2620, 19743),
 (980, 17019),
 (10703, 16946),
 (3141, 175),
 (3420, 21364),
 (2554, 16310),
 (517, 18396),
 (7107, 72),
 (333, 12626),
 (5232, 10866),
 (6604, 235),
 (8179, 15475),
 (253, 8919),
 (6023, 20697),
 (4388, 16062),
 (7324, 17812),
 (7349, 10754),
 (1303, 19332),
 (5814, 1381),
 (11346, 11624),
 (9966, 1284),
 (6442, 13064),
 (9524, 18991),
 (7651, 3141),
 (2687, 21302),
 (126, 13081),
 (5783, 979

In [74]:
user_factors

array([[0.6660371 , 0.34236557, 0.26882709, ..., 0.68222402, 0.03063557,
        0.70670297],
       [0.39174161, 0.02269831, 0.3377402 , ..., 0.13289064, 0.84806033,
        0.18483703],
       [0.89920797, 0.38919395, 0.8073527 , ..., 0.6529773 , 0.40970547,
        0.86887771],
       ...,
       [0.83954462, 0.01552441, 0.94051157, ..., 0.15806036, 0.01180685,
        0.64514884],
       [0.4723179 , 0.08434807, 0.00642827, ..., 0.42010888, 0.8705339 ,
        0.87157588],
       [0.68690211, 0.39929274, 0.07289247, ..., 0.94812402, 0.76422582,
        0.26780032]])

In [84]:
item_factors

array([[0.74811149, 0.59938846, 0.94503989, ..., 0.76167331, 0.33963907,
        0.06129092],
       [0.7935896 , 0.59504052, 0.81505204, ..., 0.89945408, 0.08941045,
        0.70660664],
       [0.09231911, 0.72370972, 0.06398683, ..., 0.29945243, 1.01394231,
        0.80106559],
       ...,
       [0.08133052, 0.12838347, 0.22624468, ..., 0.25195438, 0.3463431 ,
        0.13178942],
       [0.04306811, 0.9468368 , 0.4694165 , ..., 0.71635945, 0.81278289,
        0.05426424],
       [0.15633429, 0.05177984, 0.67115676, ..., 0.87806516, 0.26251562,
        0.44981968]])

In [99]:
true_labels = []
predicted_scores = []

for user, item in positive_samples:
    true_labels.append(1)
    predicted_scores.append(np.dot(user_factors[user], item_factors[item]))

for user, item in negative_samples:
    true_labels.append(0)
    predicted_scores.append(np.dot(user_factors[user], item_factors[item]))

In [100]:
auc_score = roc_auc_score(true_labels, predicted_scores)
print("AUC Score:", auc_score)

AUC Score: 0.5055639999999999


### Let's try to improve the score using BRP

In [98]:
for n_epoch in range(20):
  user_factors, item_factors, samples_per_second = train_one_epoch(user_factors, item_factors, learning_rate)

Iteration 12638 in 1.40 seconds. Samples per second 9013.59
Iteration 12638 in 1.37 seconds. Samples per second 9257.79
Iteration 12638 in 1.42 seconds. Samples per second 8880.57
Iteration 12638 in 1.44 seconds. Samples per second 8775.79
Iteration 12638 in 1.64 seconds. Samples per second 7729.08
Iteration 12638 in 1.33 seconds. Samples per second 9508.55
Iteration 12638 in 1.46 seconds. Samples per second 8667.39
Iteration 12638 in 1.39 seconds. Samples per second 9117.64
Iteration 12638 in 1.40 seconds. Samples per second 9045.90
Iteration 12638 in 1.45 seconds. Samples per second 8715.21
Iteration 12638 in 1.45 seconds. Samples per second 8709.20
Iteration 12638 in 1.32 seconds. Samples per second 9566.24
Iteration 12638 in 1.34 seconds. Samples per second 9444.64
Iteration 12638 in 1.34 seconds. Samples per second 9458.95
Iteration 12638 in 1.34 seconds. Samples per second 9458.78
Iteration 12638 in 1.37 seconds. Samples per second 9237.60
Iteration 12638 in 1.31 seconds. Samples

# Predictions

In [64]:
# After training, you can use the learned user and item factors for making predictions
# For example, to predict the score for a specific user and item:
user_id = 0
item_id = 1
prediction = np.dot(user_factors[user_id, :], item_factors[item_id, :])
print(f"Prediction for user {user_id} and item {item_id}: {prediction}")

Prediction for user 0 and item 1: 3.132971946898877
